# <center> Exercice : Classifiez du texte </center>

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn import metrics
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier
from collections import Counter
from scipy.sparse import csr_matrix,vstack, hstack, save_npz
import scipy as sp
import numpy as np

Le dataset RCV1 est une matrice creuse (sparse), avec 804414 échantillons et 47236 features. Les 23149 premiers échantillons constituent training set. Les derniers 781265 échantillons constituent le test set.

Je vais charger uniquement les données de training car les capacités de ma machine sont limités. J'ai aussi choisi les trois algorithmes à tester qui sont les plus rapide. J'ai essayé de tester les autres algorithmes (comme Random Forest, KNN, réseaux de neuronnes, ...), mais ces derniers consomment trop de resources et j'ai du les arrêter aprés quelques heures de traitement. Je ne les ai pas inclus dans ce notebook.

Nous commencons déja par charger les données du training set :

In [2]:
rcv1 = datasets.fetch_rcv1(subset='train', shuffle=True, random_state=0)
rcv2 = datasets.fetch_rcv1(subset='test', shuffle=True, random_state=0)

il y a 23149 échantillon et 47236 features au niveau du training set.
Le test set contient 10000 échantillon.

In [3]:
X_train = rcv1.data
y_train = rcv1.target
X_test = rcv2.data[:10000]
y_test = rcv2.target[:10000]
X_train.shape

(23149, 47236)

In [4]:
X_test

<10000x47236 sparse matrix of type '<class 'numpy.float64'>'
	with 751552 stored elements in Compressed Sparse Row format>

Il y a 84 features qui ne contiennent que des zeros. 

In [5]:
(X_train.sum(axis=0)==0).sum() #84 colonnes sur 47236 qui sont tous à 0

84

Il y a 103 labels  :

In [6]:
y_train.shape

(23149, 103)

Les noms de ces labels sont :

In [7]:
print (rcv1.target_names.tolist())

['C11', 'C12', 'C13', 'C14', 'C15', 'C151', 'C1511', 'C152', 'C16', 'C17', 'C171', 'C172', 'C173', 'C174', 'C18', 'C181', 'C182', 'C183', 'C21', 'C22', 'C23', 'C24', 'C31', 'C311', 'C312', 'C313', 'C32', 'C33', 'C331', 'C34', 'C41', 'C411', 'C42', 'CCAT', 'E11', 'E12', 'E121', 'E13', 'E131', 'E132', 'E14', 'E141', 'E142', 'E143', 'E21', 'E211', 'E212', 'E31', 'E311', 'E312', 'E313', 'E41', 'E411', 'E51', 'E511', 'E512', 'E513', 'E61', 'E71', 'ECAT', 'G15', 'G151', 'G152', 'G153', 'G154', 'G155', 'G156', 'G157', 'G158', 'G159', 'GCAT', 'GCRIM', 'GDEF', 'GDIP', 'GDIS', 'GENT', 'GENV', 'GFAS', 'GHEA', 'GJOB', 'GMIL', 'GOBIT', 'GODD', 'GPOL', 'GPRO', 'GREL', 'GSCI', 'GSPO', 'GTOUR', 'GVIO', 'GVOTE', 'GWEA', 'GWELF', 'M11', 'M12', 'M13', 'M131', 'M132', 'M14', 'M141', 'M142', 'M143', 'MCAT']


 Lorsqu'il y a plusieurs labels, comme ce dataset, nous pouvons distinguer deux cas :
##### 1. Multiclass classification : 
Dans ce cas, nous assignons un et un seul label à chaque échantillon : un un seul "1" et les reste des "0" dans chaque ligne de y.
##### 2. Multilabel classification :
Dans ce cas, nous pouvons assigner plusieurs labels à chaque échantillon : possibilité d'avoir plusieurs "1" dans chaque ligne de y.

Nous allons ici groupper les échantillons par nombre de labels :

In [8]:
Counter(((y_train.sum(axis=1).ravel()).A1))

Counter({1: 734,
         2: 6044,
         3: 10868,
         4: 2183,
         5: 1608,
         6: 1027,
         7: 361,
         8: 187,
         9: 83,
         10: 36,
         11: 13,
         12: 3,
         14: 2})

Il y uniquement 734 lignes avec un seul "1". Presque la moitier des échantillons (10868) contiennent 3 labels en même temps.

Ceci monter que notre problème est un problème : __Multi class Classification__

Regardons maintenant les labels rares, ou les nombre déchantillons avec un "1" est petit (<=30)

In [9]:
x =[]
for i in range(0,103):
    x.append([i,y_train[:,i].nonzero()[0].shape[0]])
for e in x:
    if e[1] <=30:
        print("Label %s (N° %s) : %s échantillon." %(rcv1.target_names[e[0]], e[0], e[1]))

Label E132 (N° 39) : 17 échantillon.
Label E141 (N° 41) : 12 échantillon.
Label E142 (N° 42) : 8 échantillon.
Label E312 (N° 49) : 0 échantillon.
Label E313 (N° 50) : 3 échantillon.
Label E61 (N° 57) : 15 échantillon.
Label G156 (N° 66) : 2 échantillon.
Label G159 (N° 69) : 2 échantillon.
Label GFAS (N° 77) : 6 échantillon.
Label GMIL (N° 80) : 0 échantillon.
Label GOBIT (N° 81) : 13 échantillon.
Label GTOUR (N° 88) : 23 échantillon.


Nous remarquons qu'il n'y a aucun échantillon avec les labels 49 et 80 (E312 et EGMIL)
Normalement, nous devons suuprimer ces colonnes rares de y, et voir aussi s'il y a des outliers et les supprimer si c'est le cas afin que nos algorithmes performent mieux. Mais, dans cet exercice, ce qui est demandé est de tester au moins trois algorithmes avec validation croisée sur ces données. Ainsi, nous allons laisser les données sans traitement.


In [10]:

X_train

<23149x47236 sparse matrix of type '<class 'numpy.float64'>'
	with 1757801 stored elements in Compressed Sparse Row format>

In [11]:
X_test

<10000x47236 sparse matrix of type '<class 'numpy.float64'>'
	with 751552 stored elements in Compressed Sparse Row format>

In [12]:
y_train

<23149x103 sparse matrix of type '<class 'numpy.uint8'>'
	with 73697 stored elements in Compressed Sparse Row format>

Nous allons déja tester un dummy classifier (strétégie naive) et voir le score :

In [13]:
clf = OneVsRestClassifier(DummyClassifier(random_state=0))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 49 is present in all training examples.
  str(classes[c]))
C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 80 is present in all training examples.
  str(classes[c]))


0.0

La stratégie naive ne donne pas de bons résultats.

Testons maintenant le SGDCClassifier (stochastic gradient descent) avec loss=log (logistic regression) qui est une implémentation assez rapide pour les grands datasets. 

In [14]:
clf = OneVsRestClassifier(SGDClassifier(loss='log', max_iter=5, alpha=0.00001, random_state=0))
clf.fit(X_train, y_train)


C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 49 is present in all training examples.
  str(classes[c]))
C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 80 is present in all training examples.
  str(classes[c]))


OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=0, shuffle=True,
       tol=None, verbose=0, warm_start=False),
          n_jobs=1)

In [15]:
clf.score(X_test,y_test)

0.4962

Pour la resgression logistique normale, elle prend beaucoup de temps, c'est pour ça que je ne l'ai pas inclue dans ma liste d'algorithmes.

In [16]:
clf = OneVsRestClassifier(LogisticRegression(max_iter=300, solver='sag', random_state=0))
clf.fit(X_train, y_train)

C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 49 is present in all training examples.
  str(classes[c]))
C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 80 is present in all training examples.
  str(classes[c]))


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=300, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [17]:
clf.score(X_test,y_test)

0.4156

L'implémentation SGDClassifier est beaucoup plus rapide et donne des meilleurs performances, j'ai choisi de l'inclure dans ma liste.

Nous allons tester maintenannt le SVM linéaire. J'ai choisi l'implémentation LinearSVC qui marche mieux pour les grands datasets :

In [18]:
clf = OneVsRestClassifier(LinearSVC(random_state=0))
clf.fit(X_train, y_train)

C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 49 is present in all training examples.
  str(classes[c]))
C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 80 is present in all training examples.
  str(classes[c]))


OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [19]:
clf.score(X_test,y_test)

0.5201

Le LinearSVC à donné un bon résultat.

### J'ai choisi de tester 4 algorithmes :
### 1. Le SVM linéaire (LinearSVM)
### 2. La regression logistique avec l'implémentation stochastic gradient descent (SGD, loss='log')
### 3. Le naive base algorithme (MultinomialNB)
### 4. Le SVM linèaire avec l'implémentation stochastic gradient descent (SGD, loss='hinge')

### Le choix est dicté par les ressources trés limités de ma machine et de la nature du dataset (Multi label) qui réduit le nombre de choix

In [20]:
classifiers_list = [
   OneVsRestClassifier(SGDClassifier(loss='log', random_state=0)),
   OneVsRestClassifier(MultinomialNB()),
   OneVsRestClassifier(LinearSVC(random_state=0)),
   OneVsRestClassifier(SGDClassifier(loss='hinge', random_state=0))
   #OneVsRestClassifier(RandomForestClassifier(random_state=0)),
   #MLPClassifier(random_state=0)
]

grid_params = [ 
    {
     'estimator__max_iter': [5,10,15,20],
     'estimator__alpha': [0.00001, 0.0001, 0.001, 0.01]
    },
    {
     'estimator__alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1]
    },
    {
     'estimator__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
    },
    {
     'estimator__max_iter': [5,10,15,20],
     'estimator__alpha': [0.00001, 0.0001, 0.001, 0.01]
    }
]



Je définit ici la fonction cv_search que je vais utiliser pour avoir le meilleur hyper paramétrage de mes algorithmes en utilisant la cross validation en utilisant GridSearchCV. Le score utilisé est l'accuracy. Nous aurions pu choisir d'autres indicateurs comme AUC ou F1 score.

In [21]:
def log_info(clf, X_test, y_test):
    print ("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
    print (clf.best_params_)
    # Afficher les performances correspondantes
    print ("Résultats de la validation croisée :")
    for mean, std, params in zip(clf.cv_results_['mean_test_score'], clf.cv_results_['std_test_score'], 
                                clf.cv_results_['params']
                                ):
        print ("\t%s = %0.3f (+/-%0.03f) for %r" % ('accuracy', mean,std * 2,params))
    s = clf.score(X_test,y_test)
    print ("\nSur le jeu de test : %0.3f" % s)
def cv_search(model, parameters, X_train, y_train, X_test,y_test):
    clf = GridSearchCV(model, parameters, cv=5, scoring="accuracy", n_jobs=2)
    clf.fit(X_train, y_train)
    log_info(clf, X_test, y_test)

##### Commençons par l'algorithme SGDClassifier :

In [22]:
cv_search(classifiers_list[0], grid_params[0], X_train, y_train, X_test,y_test)

C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 49 is present in all training examples.
  str(classes[c]))
C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 80 is present in all training examples.
  str(classes[c]))


Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'estimator__alpha': 1e-05, 'estimator__max_iter': 5}
Résultats de la validation croisée :
	accuracy = 0.549 (+/-0.022) for {'estimator__alpha': 1e-05, 'estimator__max_iter': 5}
	accuracy = 0.547 (+/-0.021) for {'estimator__alpha': 1e-05, 'estimator__max_iter': 10}
	accuracy = 0.545 (+/-0.021) for {'estimator__alpha': 1e-05, 'estimator__max_iter': 15}
	accuracy = 0.545 (+/-0.022) for {'estimator__alpha': 1e-05, 'estimator__max_iter': 20}
	accuracy = 0.373 (+/-0.020) for {'estimator__alpha': 0.0001, 'estimator__max_iter': 5}
	accuracy = 0.371 (+/-0.021) for {'estimator__alpha': 0.0001, 'estimator__max_iter': 10}
	accuracy = 0.369 (+/-0.021) for {'estimator__alpha': 0.0001, 'estimator__max_iter': 15}
	accuracy = 0.370 (+/-0.020) for {'estimator__alpha': 0.0001, 'estimator__max_iter': 20}
	accuracy = 0.054 (+/-0.002) for {'estimator__alpha': 0.001, 'estimator__max_iter': 5}
	accuracy = 0.053 (+/-0.002) for {'estimator__alpha': 0.001

#### Ensuite L'algorithme naive base :

In [23]:
cv_search(classifiers_list[1], grid_params[1], X_train, y_train, X_test,y_test)

C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 49 is present in all training examples.
  str(classes[c]))
C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 80 is present in all training examples.
  str(classes[c]))


Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'estimator__alpha': 0.0001}
Résultats de la validation croisée :
	accuracy = 0.396 (+/-0.015) for {'estimator__alpha': 1e-05}
	accuracy = 0.399 (+/-0.015) for {'estimator__alpha': 0.0001}
	accuracy = 0.396 (+/-0.008) for {'estimator__alpha': 0.001}
	accuracy = 0.386 (+/-0.011) for {'estimator__alpha': 0.01}
	accuracy = 0.368 (+/-0.019) for {'estimator__alpha': 0.1}
	accuracy = 0.099 (+/-0.008) for {'estimator__alpha': 1}

Sur le jeu de test : 0.306


#### Ensuite le SVM linéaire :

In [24]:
cv_search(classifiers_list[2], grid_params[2], X_train, y_train, X_test,y_test)

C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 49 is present in all training examples.
  str(classes[c]))
C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 80 is present in all training examples.
  str(classes[c]))


Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'estimator__C': 1}
Résultats de la validation croisée :
	accuracy = 0.000 (+/-0.000) for {'estimator__C': 0.001}
	accuracy = 0.124 (+/-0.011) for {'estimator__C': 0.01}
	accuracy = 0.480 (+/-0.023) for {'estimator__C': 0.1}
	accuracy = 0.578 (+/-0.023) for {'estimator__C': 1}
	accuracy = 0.560 (+/-0.017) for {'estimator__C': 10}
	accuracy = 0.527 (+/-0.015) for {'estimator__C': 100}
	accuracy = 0.484 (+/-0.013) for {'estimator__C': 1000}
	accuracy = 0.472 (+/-0.016) for {'estimator__C': 10000}

Sur le jeu de test : 0.520


#### Et enfin le SVM linéaire avec l'implémentation SGD :

In [25]:
cv_search(classifiers_list[3], grid_params[3], X_train, y_train, X_test,y_test)

C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 49 is present in all training examples.
  str(classes[c]))
C:\Continuum\Anaconda3_64\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 80 is present in all training examples.
  str(classes[c]))


Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'estimator__alpha': 1e-05, 'estimator__max_iter': 10}
Résultats de la validation croisée :
	accuracy = 0.565 (+/-0.018) for {'estimator__alpha': 1e-05, 'estimator__max_iter': 5}
	accuracy = 0.569 (+/-0.024) for {'estimator__alpha': 1e-05, 'estimator__max_iter': 10}
	accuracy = 0.568 (+/-0.018) for {'estimator__alpha': 1e-05, 'estimator__max_iter': 15}
	accuracy = 0.567 (+/-0.019) for {'estimator__alpha': 1e-05, 'estimator__max_iter': 20}
	accuracy = 0.537 (+/-0.020) for {'estimator__alpha': 0.0001, 'estimator__max_iter': 5}
	accuracy = 0.540 (+/-0.023) for {'estimator__alpha': 0.0001, 'estimator__max_iter': 10}
	accuracy = 0.540 (+/-0.022) for {'estimator__alpha': 0.0001, 'estimator__max_iter': 15}
	accuracy = 0.540 (+/-0.022) for {'estimator__alpha': 0.0001, 'estimator__max_iter': 20}
	accuracy = 0.237 (+/-0.019) for {'estimator__alpha': 0.001, 'estimator__max_iter': 5}
	accuracy = 0.238 (+/-0.018) for {'estimator__alpha': 0.00

### Nous pouvons conclure que le meilleur algorithme des quatres est le SVM Linéaire avec un score de 52%.
### Ceci peut être expliqué par la grande dimension de l'espace de définition (plus que 47000). Le nuage de points sera plus éparpillé, ce qui rend les hyperplans de séparation plus perfomants.
### Malheureusement, à cause des capacités trés réduite de ma machine locale, il n'était pas possible de tester d'autres algorithmes intéressants comme les réseaux de neuronnes et les méthodes ensemblistes comme le random forest, ni de tester sur un traning set plus grand vu qu'on dispose de plus de 800000 échantillons.
### Cependant, un score entre 50% et 55% est satisfaisant pour ce type de problèmes.

## Quelques idées supplémentaires (hors exercice) :

Nous n'avons pas effectué de feature engineering supplémentaires. Mais, nous pouvons par exemple tester une réduction de dimentions avec le PCA. Ici par exemple, j'ai utilisé TruncatedSVD qui est plus rapide pour les grands datasets :

In [26]:
from sklearn.decomposition import TruncatedSVD
pca = TruncatedSVD(n_components=3000, random_state=0)
X_res = pca.fit_transform(X_train)
pca.explained_variance_ratio_.sum()

0.7309028627684504

Nous remarquons que 3000 composants permettent d'expliquer 73% de la variance.

Il est aussi interessant supprimer les labels rares, et voir s'il y a des outliers (http://scikit-learn.org/stable/modules/outlier_detection.html) et les supprimer.

Pour la classe des problèmes de type __Multi label Classification__, il y a une bibliothéque dédiée développée au dessus de scikit learn qui s'appelle scikit-multilearn (voir le site dédié http://scikit.ml ). Nous testons ici à titre d'illustration le KNN (attention, il faudra l'installer avant. Elle n'est pas incluse dans anaconda):

In [27]:
from skmultilearn.adapt import MLkNN
clf = MLkNN(k=1)
clf.fit(X_train, y_train)

MLkNN(ignore_first_neighbours=0, k=1, s=1.0)

In [ ]:
clf.score(X_test,y_test)

0.4484